# STA 220 Data & Web Technologies for Data Analysis


### Lecture 8, 01/29/26, APIs


### Last week's topics

- APIs

### Today's topics

- Undocumented APIs

### Ressources
 - [Yolo County Health Inspections](https://inspections.myhealthdepartment.com/yolocountyeh/)

### Recap: HTTP

A response to an HTTP request always includes a status code that summarizes whether the request was successful. Wikipedia has a full [list of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). Generally,

* 200-299: Your request succeeded.
* 300-399: You need to take further action to complete the request.
* 400-499: Your request wasn't valid (you made a mistake). You've probably seen 404 before!
* 500-599: Your request failed (the server made a mistake).

Some important status codes:
- 200: OK
- 403: Access denied
- 404: Not found

#### Documented API:

In [6]:
import numpy as np
import pandas as pd
import time
import requests

https://www.whatismybrowser.com/detect/what-is-my-user-agent/

In [7]:
# Define the User-Agent header
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

response = requests.get("https://itunes.apple.com/search", headers = headers, params = {
        "term": "swift", 
        "media": "music",
        "entity": "album",
        "attribute": "artistTerm", 
        "country": "US", 
        "limit": "1"
    })

### Undocumented Web APIs

Many websites use undocumented web APIs to get data. For example:

 - [University of California Compensation](https://ucannualwage.ucop.edu/wage/)
 - [Yolo County Health Inspections](https://inspections.myhealthdepartment.com/yolocountyeh/)

You can identify these websites by looking at requests in your browser's developer tools. For Firefox and Chrome these can be accessed (Windows: <kbd>Ctrl</kbd> + <kbd>i</kbd>; MacOS: <kbd>&#8984;</kbd> + <kbd>&#8997;</kbd> + <kbd>i</kbd>).

Requests to web APIs almost always return JSON or XML data. By examining the browser requests, you can work out the endpoints and parameters, allowing you to use the API.

**CAUTION:** Web APIs that are undocumented are often undocumented for a reason. Using an undocumented API may make someone angry or get you into legal trouble! Government and quasi-government websites (like the examples above) are probably okay, as long as you cache and rate-limit your requests. For everything else, find for an alternative or get permission first.

Let's reverse engineer the webiste so that we can get the data.

In [8]:
!pip install requests_cache

Check the [docs](https://requests.readthedocs.io/en/latest/api/?highlight=post#requests.post) for `requests`!

In [9]:
import requests_cache
requests_cache.install_cache("../output/lecture8a")

In [10]:
url = 'https://inspections.myhealthdepartment.com/yolocountyeh/search'

In [11]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
result = requests.get(url, headers = headers, params = {
    'searchStr': "Manna"
})
result.raise_for_status()

In [12]:
result.url

'https://inspections.myhealthdepartment.com/yolocountyeh/search?searchStr=Manna'

In [13]:
result.content

b'<!doctype html>\n<html lang="en" data-wf-page="5c477d7234329c8ae83693a0" data-wf-site="5c477d7234329c39c036939f">\n<head>\n\t<meta charset="utf-8">\n\t<title>Search | Yolo County California | My Health Department</title>\n\t<meta content="width=device-width, initial-scale=1" name="viewport">\n\t<meta content="Webflow" name="generator">\n\t<!-- Sentry Integration -->\n\t<script\n\t\tsrc="https://d1x0oaju7ljwc0.cloudfront.net/sentry/bundle.tracing.replay.feedback.min.js"\n\t\tintegrity="sha384-Yy2UXIFrWRfe56w1BuJ8/pgltHwWyYP4Q7dYKueJ/c6RG8B/bPJmGv+TBTQSuTSv"\n\t\tcrossorigin="anonymous"\n\t></script>\n\t<script src="/js/sentry.js"></script>\n\t<!-- include Flatpickr JS and CSS for datepicker-->\n\t<link href="css/vendor/flatpickr/flatpickr.css" rel="stylesheet" type="text/css">\n\t<script src="js/vendor/flatpickr/flatpickr.js" type="text/javascript"></script>\n\t<link href="css/myhd-full.min.css" rel="stylesheet" type="text/css">\n\t<script src="https://use.typekit.net/ksy3qdn.js" type

Okay, but how to find an undocumented API?

A __step by step__ guide:
- Open the [website](https://inspections.myhealthdepartment.com/yolocountyeh/).
- Inspect the page either by `right-click + Inspect` or using `cmd + i`.
- Click on the tab `Network`.
- Interact with the page to enforce the API request (e.g. Change the date).
- (Optional): Filter the urls.
- Look for a POST/GET method whose return is, ideally, in `json`.
- Click on the element.
- On the right-hand side: select the field `response` to see what is returned.
- On the right-hand side: select the field `request` to see what how to access it.
- Add these parameters to your request in Python.

In [14]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:147.0) Gecko/20100101 Firefox/147.0'
}

json_data = {
    'data': {
        'path': 'yolocountyeh',
        'programName': '',
        'filters': {
            'date': '2026-01-01 to 2026-01-27',
            'purpose': 'Routine',
        },
        'start': 0,
        'count': 20,
        'searchStr': '',
        'lat': 0,
        'lng': 0,
        'sort': {},
    },
    'task': 'searchInspections',
}

response = requests.post('https://inspections.myhealthdepartment.com/', 
                         headers=headers, json=json_data)

In [15]:
response.raise_for_status()

In [16]:
response.json()

[{'nick': 'ycc',
  'inspectionID': '259A6A34-CA12-448D-B2A8-437B31002A0D',
  'inspectionDate': '2026-01-27T00:00:00.000Z',
  'score': None,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': 'NO VIOLATIONS OBSERVED.  \n\nGREAT JOB!!!',
  'StartTime': '1899-12-30T13:00:00.000Z',
  'establishmentName': 'GALLERY COFFEE ROASTERS',
  'addressLine1': '420 1st St ',
  'addressLine2': '',
  'city': 'Woodland',
  'state': 'CA',
  'zip': '95695-4024',
  'permitID': 'E451D396-434A-4A35-8F78-A50D11813154',
  'progIdent': None,
  'cfg_permit_typeID': '130D172C-232E-45EE-B256-2623105B6119',
  'INSP_PURPOSEID': 'Routine',
  'permitType': 'Restaurant 0-25 seats, Risk Category (RC) 1',
  'programName': 'Food'},
 {'nick': 'ycc',
  'inspectionID': '847795F2-72FC-42EF-93B3-F3DB7DDBAC14',
  'inspectionDate': '2026-01-27T00:00:00.000Z',
  'score': 0,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': 'A routine inspection was completed, no violations observed.\n\nThe time i

https://curlconverter.com/

In [17]:
with open("../data/cookies.json", "r") as json_file:
    cookies = json.load(json_file)

In [18]:
type(cookies)

dict

In [19]:
len(cookies)

3

In [20]:
import requests

#cookies = {
#    '_ga': 'PLACEHOLDER',
#    '_ga_Y6L8QZ2CVV': 'PLACEHOLDER',
#    'connect.sid': 'PLACEHOLDER',
#}

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:147.0) Gecko/20100101 Firefox/147.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-type': 'application/json',
    'Origin': 'https://inspections.myhealthdepartment.com',
    'Connection': 'keep-alive',
    'Referer': 'https://inspections.myhealthdepartment.com/yolocountyeh?task=searchInspections',
    # 'Cookie': '_ga=GA1.2.1803900147.1769190027; _ga_Y6L8QZ2CVV=GS2.2.s1769190027$o1$g0$t1769190027$j60$l0$h0; connect.sid=s%3Af5zz32iranx9ung90qjixtl20bo1kk8xrkhobubsl6x1s7s4rgn5n98tfigm.lAZKUUh6EVlz1W%2Fycz1smRbiy3G3YUxYOBTZ7hh%2FGMs',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Priority': 'u=0',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

json_data = {
    'data': {
        'path': 'yolocountyeh',
        'programName': '',
        'filters': {
            'date': '2026-01-01 to 2026-01-25',
            'purpose': 'Routine',
        },
        'start': 26,
        'count': 40,
        'searchStr': '',
        'lat': 0,
        'lng': 0,
        'sort': {},
    },
    'task': 'searchInspections',
}

response = requests.post('https://inspections.myhealthdepartment.com/', 
#                         cookies=cookies, 
                         headers=headers, 
                         json=json_data)

In [21]:
#with open("../data/cookies.json", "w") as json_file:
#    json.dump(cookies, json_file, indent=4)

In [22]:
response.raise_for_status()

In [23]:
data = response.json()

In [24]:
data

[{'nick': 'ycc',
  'inspectionID': '078E3834-65E7-4180-9168-BC8DE42C2407',
  'inspectionDate': '2026-01-21T00:00:00.000Z',
  'score': None,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': 'shahsdavis@gmail.com ',
  'StartTime': '1899-12-30T13:00:00.000Z',
  'establishmentName': "SHAH'S HALAL DAVIS INC. DBA SHA'S HALAL FOOD",
  'addressLine1': '865 Russell Blvd Ste 120 ',
  'addressLine2': '',
  'city': 'Davis',
  'state': 'CA',
  'zip': '95616-3426',
  'permitID': '321D7183-8413-4E96-BF82-C81BE0C3BD14',
  'progIdent': '',
  'cfg_permit_typeID': 'C94A3B79-8277-4B03-8C28-99C7F1C9E5F7',
  'INSP_PURPOSEID': 'Routine',
  'permitType': 'RESTAURANT OVER 650 SQ FT - YEAR PERMIT',
  'programName': 'Food'},
 {'nick': 'ycc',
  'inspectionID': 'B908E411-C6D8-4130-9FC1-F96FB9F7E342',
  'inspectionDate': '2026-01-21T00:00:00.000Z',
  'score': None,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': 'Time noted on the front page includes drivetime, inspection, and

In [25]:
df = pd.DataFrame(data)

In [26]:
df.head()

,nick,inspectionID,inspectionDate,score,inspectionType,comments,StartTime,establishmentName,addressLine1,addressLine2,city,state,zip,permitID,progIdent,cfg_permit_typeID,INSP_PURPOSEID,permitType,programName
0,ycc,078E3834-65E7-4180-9168-BC8DE42C2407,2026-01-21T00:00:00.000Z,NaN,Retail Food Routine Inspection,shahsdavis@gmail.com,1899-12-30T13:00:00.000Z,SHAH'S HALAL DAVIS INC. DBA SHA'S HALAL FOOD,865 Russell Blvd Ste 120,,Davis,CA,95616-3426,321D7183-8413-4E96-BF82-C81BE0C3BD14,,C94A3B79-8277-4B03-8C28-99C7F1C9E5F7,Routine,RESTAURANT OVER 650 SQ FT - YEAR PERMIT,Food
1,ycc,B908E411-C6D8-4130-9FC1-F96FB9F7E342,2026-01-21T00:00:00.000Z,NaN,Retail Food Routine Inspection,Time noted on the front page includes drivetim...,1899-12-30T11:00:00.000Z,LA PLAZITA - MADISON,17780 Cr 89,,Madison,CA,95653,53F46C47-589B-4885-AF34-E8B92E5F6DFD,None,3A5D5ECC-2853-44D1-94D5-9BE503011763,Routine,"Restaurant 26-49 seats, RC 3",Food
2,ycc,1C9887EC-E8EC-4E68-95D0-51A049DA1A85,2026-01-21T00:00:00.000Z,0.0,Retail Food Routine Inspection,,1899-12-30T10:45:00.000Z,EL MACERO COUNTRY CLUB,44571 Clubhouse Dr,,El Macero,CA,95618-1073,A0B6AF89-BEBB-4783-8282-78CE58650827,None,AE3088E4-CCB8-4615-9CB0-DC3B15CA961F,Routine,RESTAURANT/BAR COMBINATION - YEAR PERMIT,Food
3,ycc,7C53DC08-5F4A-4B53-8C1B-AC24A167A568,2026-01-21T00:00:00.000Z,0.0,Retail Food Routine Inspection,A routine inspection was completed. The facili...,1899-12-30T12:00:00.000Z,LA JALICIENSE NO 2,91 W Court St Ste A,,Woodland,CA,95695-3007,845CE087-7795-4EBD-A9F2-CD0D83D70B27,None,329FB2A3-BC30-43C6-B909-F2626436D432,Routine,"Bakery less than 2,000 square feet, RC2",Food
4,ycc,094F20E0-3C51-4BEA-8BA5-8975527009DD,2026-01-20T00:00:00.000Z,0.0,Retail Food Routine Inspection,A routine inspection was completed. The time i...,1899-12-30T10:45:00.000Z,WAYSIDE MARKET,9696 Locust St,,KNIGHTS LANDING,CA,95645,898CD7BC-632E-4967-B9BF-7B6B75721A77,None,49C7792C-12AF-4C8F-B59A-CEC24FA22144,Routine,MARKET W/1 FOOD PREP/PROCESS (MARKET/DELI/ICE),Food


In [27]:
df.shape

(25, 19)

In [28]:
def get_page(page_nr):

    json_data = {
        'data': {
            'path': 'yolocountyeh',
            'programName': '',
            'filters': {
                'date': '2025-12-01 to 2026-01-25',
                'purpose': 'Routine',
            },
            'start': page_nr,
            'count': 25,
            'searchStr': '',
            'lat': 0,
            'lng': 0,
            'sort': {},
        },
        'task': 'searchInspections',
    }

    response = requests.post('https://inspections.myhealthdepartment.com/', 
                         cookies=cookies, headers=headers, json=json_data)
    response.raise_for_status()
    return(response.json())

In [29]:
import time

In [30]:
data = []
for i in range(20):
    time.sleep(1) # SLOW DOWN the process!
    tmp = get_page(25*i)
    if not len(tmp):
        print('Stopped at page ' + str(i))
        break
    data += tmp

Stopped at page 7


In [31]:
data

[{'nick': 'ycc',
  'inspectionID': 'BEA0E8F3-4D2D-41A5-9470-FAE452250500',
  'inspectionDate': '2026-01-23T00:00:00.000Z',
  'score': 0,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': 'No violation observed at the time of inspection.',
  'StartTime': '1899-12-30T13:15:00.000Z',
  'establishmentName': 'SWEET & SHAVERY',
  'addressLine1': '210 E St ',
  'addressLine2': '',
  'city': 'Davis',
  'state': 'CA',
  'zip': '95616-4522',
  'permitID': '6EB917DA-A458-4D55-AD93-2D75F6A65145',
  'progIdent': None,
  'cfg_permit_typeID': '130D172C-232E-45EE-B256-2623105B6119',
  'INSP_PURPOSEID': 'Routine',
  'permitType': 'Restaurant 0-25 seats, Risk Category (RC) 1',
  'programName': 'Food'},
 {'nick': 'ycc',
  'inspectionID': '0524268C-DD06-4406-8FFC-709375B38A67',
  'inspectionDate': '2026-01-23T00:00:00.000Z',
  'score': None,
  'inspectionType': 'Retail Food Routine Inspection',
  'comments': '',
  'StartTime': '1899-12-30T12:00:00.000Z',
  'establishmentName': 'YOCHA DEH

In [32]:
df = pd.DataFrame(data)
df.head()

,nick,inspectionID,inspectionDate,score,inspectionType,comments,StartTime,establishmentName,addressLine1,addressLine2,city,state,zip,permitID,progIdent,cfg_permit_typeID,INSP_PURPOSEID,permitType,programName
0,ycc,BEA0E8F3-4D2D-41A5-9470-FAE452250500,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T13:15:00.000Z,SWEET & SHAVERY,210 E St,,Davis,CA,95616-4522,6EB917DA-A458-4D55-AD93-2D75F6A65145,None,130D172C-232E-45EE-B256-2623105B6119,Routine,"Restaurant 0-25 seats, Risk Category (RC) 1",Food
1,ycc,0524268C-DD06-4406-8FFC-709375B38A67,2026-01-23T00:00:00.000Z,NaN,Retail Food Routine Inspection,,1899-12-30T12:00:00.000Z,YOCHA DEHE GOLF CLUB - FOOD,14455 State Highway 16,,Brooks,CA,95606-9707,7ECF4634-BF6F-4963-AE0B-2FD7E86A4234,None,E24FBE78-E62B-4246-8A3C-F32C304C5D42,Routine,"Restaurant 0-25 seats, RC 2",Food
2,ycc,88B2441C-3B45-49EA-BCCA-F64BCE3E18FA,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T12:45:00.000Z,KOBE MINI MART,213 E St Ste A,,Davis,CA,95616-4573,8535210B-F305-421C-A216-C042C2637DED,None,49C7792C-12AF-4C8F-B59A-CEC24FA22144,Routine,MARKET W/1 FOOD PREP/PROCESS (MARKET/DELI/ICE),Food
3,ycc,32A4027B-4C9A-4198-921A-1707EFED3614,2026-01-23T00:00:00.000Z,NaN,Retail Food Routine Inspection,Time noted on the first page includes drivetim...,1899-12-30T12:45:00.000Z,TABER RANCH VINEYARD & EVENT CENTER,16628 CR 81,,CAPAY,CA,95607,4A6494FD-B161-429A-8AB6-1AA873EF02A2,None,2461793F-33B2-48C0-A357-3A5CCE21616B,Routine,EVENT CENTER,Food
4,ycc,C69F4DA9-BBD4-4411-9615-515D6C1D0ED0,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T14:00:00.000Z,TEMPLE COFFEE,239 G St,,Davis,CA,95616-4516,F34D3ED0-D607-4B97-97C6-87EF7763980A,None,130D172C-232E-45EE-B256-2623105B6119,Routine,"Restaurant 0-25 seats, Risk Category (RC) 1",Food


In [33]:
df.describe(include=['object'])

,nick,inspectionID,inspectionDate,inspectionType,comments,StartTime,establishmentName,addressLine1,addressLine2,city,state,zip,permitID,progIdent,cfg_permit_typeID,INSP_PURPOSEID,permitType,programName
count,167,167,167,167,167,167,167,167,166,167,167,167,167,11,167,167,167,167
unique,1,164,29,4,77,33,157,151,4,16,1,128,164,1,39,1,39,4
top,ycc,424A02AF-0107-4681-8B76-CA1DEBEA38D8,2026-01-21T00:00:00.000Z,Retail Food Routine Inspection,,1899-12-30T12:00:00.000Z,TOGO'S,2051 Bronze Star Dr Ste 100,,Davis,CA,95616-4654,7CF022EE-E5E0-408A-894D-1EB82FAAA8C4,,C94A3B79-8277-4B03-8C28-99C7F1C9E5F7,Routine,RESTAURANT OVER 650 SQ FT - YEAR PERMIT,Food
freq,167,2,18,147,72,29,2,2,163,73,167,5,2,11,21,167,21,150


In [34]:
df[df['city'] == 'Davis']

,nick,inspectionID,inspectionDate,score,inspectionType,comments,StartTime,establishmentName,addressLine1,addressLine2,city,state,zip,permitID,progIdent,cfg_permit_typeID,INSP_PURPOSEID,permitType,programName
0,ycc,BEA0E8F3-4D2D-41A5-9470-FAE452250500,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T13:15:00.000Z,SWEET & SHAVERY,210 E St,,Davis,CA,95616-4522,6EB917DA-A458-4D55-AD93-2D75F6A65145,None,130D172C-232E-45EE-B256-2623105B6119,Routine,"Restaurant 0-25 seats, Risk Category (RC) 1",Food
2,ycc,88B2441C-3B45-49EA-BCCA-F64BCE3E18FA,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T12:45:00.000Z,KOBE MINI MART,213 E St Ste A,,Davis,CA,95616-4573,8535210B-F305-421C-A216-C042C2637DED,None,49C7792C-12AF-4C8F-B59A-CEC24FA22144,Routine,MARKET W/1 FOOD PREP/PROCESS (MARKET/DELI/ICE),Food
4,ycc,C69F4DA9-BBD4-4411-9615-515D6C1D0ED0,2026-01-23T00:00:00.000Z,0.0,Retail Food Routine Inspection,No violation observed at the time of inspection.,1899-12-30T14:00:00.000Z,TEMPLE COFFEE,239 G St,,Davis,CA,95616-4516,F34D3ED0-D607-4B97-97C6-87EF7763980A,None,130D172C-232E-45EE-B256-2623105B6119,Routine,"Restaurant 0-25 seats, Risk Category (RC) 1",Food
5,ycc,63878BBE-0FEC-4972-82F1-DE11442F4F06,2026-01-23T00:00:00.000Z,NaN,Retail Food Routine Inspection,,1899-12-30T11:00:00.000Z,ROYAL KABOB AND FALAFEL,640 W Covell Blvd,,Davis,CA,95616-1064,62BE8976-60EE-4B00-9FF1-EDA04834A81C,None,E24FBE78-E62B-4246-8A3C-F32C304C5D42,Routine,"Restaurant 0-25 seats, RC 2",Food
9,ycc,07132A05-4C5E-4F00-980A-2F7252183164,2026-01-22T00:00:00.000Z,0.0,Retail Food Routine Inspection,,1899-12-30T12:30:00.000Z,OLIVE DRIVE SHELL,1010 Olive Dr,,Davis,CA,95616-4721,E29C9F2D-5781-4EBE-A703-C480A0386348,None,3908BFBB-683F-4509-8D82-18636E382CE5,Routine,"Retail Food Markets 2,000-4,999 square feet, RC1",Food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,ycc,7DFF5355-5CD8-41B0-9E1E-DC9EA01BA256,2025-12-04T00:00:00.000Z,NaN,Retail Food Routine Inspection,,1899-12-30T12:15:00.000Z,FAST & EASY MART #33 - FOOD,1601 Research Park Dr,,Davis,CA,95618-6159,7DCDBDB5-DF7E-44FE-81CB-BCAF77A53B04,None,2939927A-FB66-4C15-9334-C4ED4FE1AE1F,Routine,"Retail Food Markets less than 2,000 square fee...",Food
159,ycc,9B1F79F1-63EF-4194-A775-DFFD190ACC9D,2025-12-03T00:00:00.000Z,NaN,Retail Food Routine Inspection,,1899-12-30T13:00:00.000Z,TIM'S HAWAIIAN BBQ,247 3rd St,,Davis,CA,95616-4524,E0BA8259-7EC1-44C4-8943-D005CEC7FB53,None,904A5E18-0946-499C-8DF3-B8C23EBC3CB8,Routine,"Restaurant 26-49 seats, RC 2",Food
160,ycc,9122F376-4585-4C05-B2CC-24E6633653AD,2025-12-03T00:00:00.000Z,NaN,Retail Food Routine Inspection,,1899-12-30T11:30:00.000Z,3RD & U CAFE,223 3rd St,,Davis,CA,95616-4501,F5317037-F065-4160-8363-F89D4512D5E1,None,153791A7-6FFF-4287-9D24-80E1D3AAD718,Routine,"Restaurant 50-149 seats, RC 2",Food
162,ycc,0A4B749F-B897-42B1-9D2E-57889F98FBA5,2025-12-02T00:00:00.000Z,NaN,Retail Food Routine Inspection,Note - Facility will be undergoing a kitchen r...,1899-12-30T10:30:00.000Z,PEREGRINE SCHOOL,2650 Lillard Dr,,Davis,CA,95618-6130,CEED95EF-5B21-4970-A28B-27BB7F6BBAC0,None,F1198CAF-5637-451B-8B03-91584AD6761E,Routine,School / Institutional Food Service Facility- ...,Food


In [35]:
df.shape

(167, 19)

Let's investigate this further:

Get all inspections for one restaurant.

In [42]:
def get_rest_visits(permitId):
    
    response = requests.get(
        'https://inspections.myhealthdepartment.com/yolocountyeh/permit',
        params={
            'permitID': permitId   
        },
        headers=headers
    )
    response.raise_for_status()
    return(response.content)

In [43]:
df['permitID'][0]

'6EB917DA-A458-4D55-AD93-2D75F6A65145'

In [44]:
page = get_rest_visits(df['permitID'][0])

In [45]:
import lxml.html as lx

html = lx.fromstring(page)
html

<Element html at 0x12f8fa260>

In [46]:
links = html.xpath("//a[contains(@class, 'inspection-listing-score w-button')]/@href")

In [47]:
for l in links:
    print(l)

/yolocountyeh/inspection/?inspectionID=BEA0E8F3-4D2D-41A5-9470-FAE452250500
/yolocountyeh/inspection/?inspectionID=305CDEF1-AE6C-4B72-BF5B-3AB8A10570A9


In [48]:
base_url = "https://inspections.myhealthdepartment.com"

In [49]:
response = requests.get(base_url + links[0], headers=headers)
response.raise_for_status()
html = lx.fromstring(response.content)

In [53]:
response.url

'https://inspections.myhealthdepartment.com/yolocountyeh/inspection/?inspectionID=BEA0E8F3-4D2D-41A5-9470-FAE452250500'

In [54]:
score_btn = html.xpath("//span[contains(@class, 'inspection-score-v2 w-button')]/span")[0]

In [55]:
score = score_btn.xpath("//strong")[1].text

In [56]:
def get_score_from_inspection(inspectionID):
    response = requests.get(
        'https://inspections.myhealthdepartment.com/yolocountyeh/inspection/',
        params={
            'inspectionID': inspectionID   
        },
        headers=headers
    )
    response.raise_for_status()
    html = lx.fromstring(response.content)
    score_btn = html.xpath("//span[contains(@class, 'inspection-score-v2 w-button')]/span")[0]
    score = score_btn.xpath("//strong")[1].text
    
    return(score)

In [57]:
get_score_from_inspection("BEA0E8F3-4D2D-41A5-9470-FAE452250500")

'0'

In [58]:
import re
def extract_inspectionID(link):
    match = re.search(r'=(.*)', link)
    if match:
        return(match.group(1))
    else:
        return None

In [59]:
extract_inspectionID(links[0])

'BEA0E8F3-4D2D-41A5-9470-FAE452250500'

In [60]:
get_score_from_inspection("BEA0E8F3-4D2D-41A5-9470-FAE452250500")

'0'

In [84]:
base_url = "https://inspections.myhealthdepartment.com/tennessee"

In [88]:
def get_page(page_nr, county_name = "tennessee"):

    json_data = {
        'data': {
            'path': county_name,
            'programName': '',
            'filters': {
                'date': '2026-01-01 to 2026-01-25',
                'purpose': '',
            },
            'start': page_nr,
            'count': 25,
            'searchStr': '',
            'lat': 0,
            'lng': 0,
            'sort': {},
        },
        'task': 'searchInspections',
    }

    response = requests.post('https://inspections.myhealthdepartment.com/', 
                         #cookies=cookies, 
                        headers=headers, json=json_data)
    response.raise_for_status()
    result = response.json()
    if isinstance(result, list):
        return result
    else:
        return None     

In [89]:
get_page(1)

[{'nick': 'stdh',
  'inspectionID': '8BA844A0-25BD-4C48-8E82-D1DDB65D4695',
  'inspectionDate': '2026-01-23T00:00:00.000Z',
  'recommendation': '',
  'score': 100,
  'inspectionType': 'Food Service Establishment Inspection',
  'purpose': 'Routine',
  'comments': 'Approved mobile to operate. Will send copy of permit. Provided food safety fact sheets for op\n\nDiscussed proper hand washing, ware washing, food source, holding and cook temps, cooling when applicable, employee hygiene, employee health, demonstration of knowledge, and storage and use of toxic items. Food Establishment Regulations can be found at https://publications.tnsosfiles.com/rules/1200/1200-23/1200-23-01.20150716.pdf. Please be sure you and all food handling employees are familiar with these regulations. Guidance/Educational documents can be found at https://www.tn.gov/health/health-program-areas/eh/eh-foodlaw.html. These are an excellent resource to help reduce the risk of a foodborne illness. If you have any question

In [90]:
data = []
for i in range(5):
    time.sleep(1) # SLOW DOWN the process!
    tmp = get_page(25*i)
    if not tmp:
        print('Stopped at page ' + str(i))
        break
    data += tmp

In [91]:
data

[{'nick': 'stdh',
  'inspectionID': 'F14D9D7C-6040-4579-ACCB-864AD185037F',
  'inspectionDate': '2026-01-23T00:00:00.000Z',
  'recommendation': '',
  'score': 100,
  'inspectionType': 'Public Swimming Pools',
  'purpose': 'Routine',
  'comments': '',
  'timein': '1899-12-30T09:00:00.000Z',
  'establishmentName': 'Hamilton Family YMCA',
  'addressLine1': '7430 Shallowford Rd.',
  'addressLine2': '',
  'city': 'Chattanooga',
  'state': 'TN',
  'zip': '37421',
  'scoreDisplay': '',
  'permitID': 'F6556C9D-1982-419B-B96E-164D9F9831B7',
  'permitType': 'Type A- General public and institutional pools',
  'programName': 'Public Swimming Pool',
  'programCode': '690'},
 {'nick': 'stdh',
  'inspectionID': '8BA844A0-25BD-4C48-8E82-D1DDB65D4695',
  'inspectionDate': '2026-01-23T00:00:00.000Z',
  'recommendation': '',
  'score': 100,
  'inspectionType': 'Food Service Establishment Inspection',
  'purpose': 'Routine',
  'comments': 'Approved mobile to operate. Will send copy of permit. Provided foo

In [92]:
df = pd.DataFrame(data)
df.head()

,nick,inspectionID,inspectionDate,recommendation,score,inspectionType,purpose,comments,timein,establishmentName,addressLine1,addressLine2,city,state,zip,scoreDisplay,permitID,permitType,programName,programCode
0,stdh,F14D9D7C-6040-4579-ACCB-864AD185037F,2026-01-23T00:00:00.000Z,,100.0,Public Swimming Pools,Routine,,1899-12-30T09:00:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F6556C9D-1982-419B-B96E-164D9F9831B7,Type A- General public and institutional pools,Public Swimming Pool,690
1,stdh,8BA844A0-25BD-4C48-8E82-D1DDB65D4695,2026-01-23T00:00:00.000Z,,100.0,Food Service Establishment Inspection,Routine,Approved mobile to operate. Will send copy of ...,1899-12-30T09:45:00.000Z,La Media Naranja LLC #2 Mobile FSE,7826 Santos Dr,,Murfreesboro,TN,37129,,53A4904B-BFF6-4E5F-BF43-79253A7BD020,Commercial Food <51 (Mobile),Food Service Establishment,605
2,stdh,C47AF1AD-1C7B-4AC4-9C4A-2D320A035257,2026-01-23T00:00:00.000Z,,100.0,Public Swimming Pools,Routine,,1899-12-30T09:10:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F9B53208-596A-40AB-AB59-D953C102B970,"Type D- Whirlpools, hot tubs",Public Swimming Pool,690
3,stdh,292318AF-BD49-4DE0-B891-E5E42590424E,2026-01-23T00:00:00.000Z,,100.0,Tattoo Studios Inspection,Routine,Establishment is very clean and organized!,1899-12-30T13:00:00.000Z,Got Ink? #615,8204 Florence Rd,,Smyrna,TN,37167,,F31C828F-71DC-4DDA-A56B-895DA3DFE22C,Tattoo Studios,Tattoo Studios,665
4,stdh,9C54FBC1-B6E3-4B22-816A-B01E8406E1A8,2026-01-23T00:00:00.000Z,,92.0,Food Service Establishment Inspection,Routine,,1899-12-30T13:34:00.000Z,Subway #2958,610 Dolly Parton Pkwy.,,Sevierville,TN,37862,,3691D738-F327-4327-8D16-7E5E28BB76BC,Commercial Food <51,Food Service Establishment,605


In [93]:
df.shape

(125, 20)

In [94]:
df.describe(include=['object'])

,nick,inspectionID,inspectionDate,recommendation,inspectionType,purpose,comments,timein,establishmentName,addressLine1,addressLine2,city,state,zip,scoreDisplay,permitID,permitType,programName,programCode
count,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125,125
unique,1,106,1,2,8,2,59,82,104,101,11,43,2,66,1,106,20,8,8
top,stdh,114BD410-C414-4886-A191-6D8C47984331,2026-01-23T00:00:00.000Z,,Food Service Establishment Inspection,Routine,,1899-12-30T11:40:00.000Z,Roosevelt Elementary Cafeteria,1051 Lake St.,,Nashville,TN,37203,,F1F4B999-757B-4822-B59A-95BEC1BB1113,Commercial Food <51,Food Service Establishment,605
freq,125,3,125,121,94,80,57,3,3,4,114,23,124,8,125,3,34,94,94


In [95]:
def get_score_from_inspection(inspectionID, base_url):
    response = requests.get(
        base_url + "/inspection/",
        params={
            'inspectionID': inspectionID   
        },
        headers=headers
    )
    response.raise_for_status()
    html = lx.fromstring(response.content)
    try:
        score_btn = html.xpath("//span[contains(@class, 'inspection-score-v2 w-button')]/span")[0]
        score = score_btn.xpath("//strong")[1].text
    except:
        return None
    else:   
        return(score)

In [96]:
df['score'] = None

In [97]:
df

,nick,inspectionID,inspectionDate,recommendation,score,inspectionType,purpose,comments,timein,establishmentName,addressLine1,addressLine2,city,state,zip,scoreDisplay,permitID,permitType,programName,programCode
0,stdh,F14D9D7C-6040-4579-ACCB-864AD185037F,2026-01-23T00:00:00.000Z,,None,Public Swimming Pools,Routine,,1899-12-30T09:00:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F6556C9D-1982-419B-B96E-164D9F9831B7,Type A- General public and institutional pools,Public Swimming Pool,690
1,stdh,8BA844A0-25BD-4C48-8E82-D1DDB65D4695,2026-01-23T00:00:00.000Z,,None,Food Service Establishment Inspection,Routine,Approved mobile to operate. Will send copy of ...,1899-12-30T09:45:00.000Z,La Media Naranja LLC #2 Mobile FSE,7826 Santos Dr,,Murfreesboro,TN,37129,,53A4904B-BFF6-4E5F-BF43-79253A7BD020,Commercial Food <51 (Mobile),Food Service Establishment,605
2,stdh,C47AF1AD-1C7B-4AC4-9C4A-2D320A035257,2026-01-23T00:00:00.000Z,,None,Public Swimming Pools,Routine,,1899-12-30T09:10:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F9B53208-596A-40AB-AB59-D953C102B970,"Type D- Whirlpools, hot tubs",Public Swimming Pool,690
3,stdh,292318AF-BD49-4DE0-B891-E5E42590424E,2026-01-23T00:00:00.000Z,,None,Tattoo Studios Inspection,Routine,Establishment is very clean and organized!,1899-12-30T13:00:00.000Z,Got Ink? #615,8204 Florence Rd,,Smyrna,TN,37167,,F31C828F-71DC-4DDA-A56B-895DA3DFE22C,Tattoo Studios,Tattoo Studios,665
4,stdh,9C54FBC1-B6E3-4B22-816A-B01E8406E1A8,2026-01-23T00:00:00.000Z,,None,Food Service Establishment Inspection,Routine,,1899-12-30T13:34:00.000Z,Subway #2958,610 Dolly Parton Pkwy.,,Sevierville,TN,37862,,3691D738-F327-4327-8D16-7E5E28BB76BC,Commercial Food <51,Food Service Establishment,605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,stdh,ADAC61BD-507E-40EA-BB09-FB101AE8FA28,2026-01-23T00:00:00.000Z,Approval,None,Child Care Facilities Inspection,Routine,Water temp at hand sink: 109 degrees. \nkim719...,1899-12-30T10:10:00.000Z,Troy Head Start,409 S. Maple St.,,Troy,TN,38260,,80212FB3-D403-438E-8C33-EE082E9A2BB6,Child Care Center,Child Care Facility,631
121,stdh,91B2D27E-28DD-4804-836C-4CFBCFF741E6,2026-01-23T00:00:00.000Z,,None,Body Piercing Studios Inspection,Routine,Verified establishment has current permit and ...,1899-12-30T13:31:00.000Z,Pinnacle Piercing,607 Sweetwater-Vonroe Road,,Sweetwater,TN,37874,,B03916D6-BDEA-47A8-BF81-662C374A867D,Body Piercing Studios,Body Piercing Studios,670
122,stdh,2EBAB81B-88E2-4183-9FA7-143C752FF94F,2026-01-23T00:00:00.000Z,,None,Food Service Establishment Inspection,Follow-Up,,1899-12-30T11:59:00.000Z,W.C. Taco Shop Wilma,1725 Wilma Rudolph Blvd.,Suite E,Clarksville,TN,37040,,C1DDA82D-38C5-47E0-986C-30609D3E88A9,Commercial Food <51,Food Service Establishment,605
123,stdh,1703EF20-3CA0-4046-858E-7E99FEE3C72C,2026-01-23T00:00:00.000Z,,None,Food Service Establishment Inspection,Follow-Up,Priority violations #6 & #21 were corrected on...,1899-12-30T12:40:00.000Z,The Salvation Army Kitchen,200 Ashe St.,,Johnson City,TN,37604,,5AF3BE77-C994-438E-A221-9055BFF53C95,Commercial Food 51+,Food Service Establishment,605


In [98]:
df['inspectionID']

0      F14D9D7C-6040-4579-ACCB-864AD185037F
1      8BA844A0-25BD-4C48-8E82-D1DDB65D4695
2      C47AF1AD-1C7B-4AC4-9C4A-2D320A035257
3      292318AF-BD49-4DE0-B891-E5E42590424E
4      9C54FBC1-B6E3-4B22-816A-B01E8406E1A8
                       ...                 
120    ADAC61BD-507E-40EA-BB09-FB101AE8FA28
121    91B2D27E-28DD-4804-836C-4CFBCFF741E6
122    2EBAB81B-88E2-4183-9FA7-143C752FF94F
123    1703EF20-3CA0-4046-858E-7E99FEE3C72C
124    57EFABC5-55F2-4F4B-A6F0-8FE574023982
Name: inspectionID, Length: 125, dtype: object

In [99]:
df['score'] = None
for key, val in df['inspectionID'].items():
    time.sleep(1)
    df.loc[key,'score'] = get_score_from_inspection(val, base_url)

HTTPError: 403 Client Error: Forbidden for url: https://inspections.myhealthdepartment.com/tennessee/inspection/?inspectionID=9687306A-3C3F-41B9-B058-02623C7B12A2

In [100]:
df.head()

,nick,inspectionID,inspectionDate,recommendation,score,inspectionType,purpose,comments,timein,establishmentName,addressLine1,addressLine2,city,state,zip,scoreDisplay,permitID,permitType,programName,programCode
0,stdh,F14D9D7C-6040-4579-ACCB-864AD185037F,2026-01-23T00:00:00.000Z,,100,Public Swimming Pools,Routine,,1899-12-30T09:00:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F6556C9D-1982-419B-B96E-164D9F9831B7,Type A- General public and institutional pools,Public Swimming Pool,690
1,stdh,8BA844A0-25BD-4C48-8E82-D1DDB65D4695,2026-01-23T00:00:00.000Z,,100,Food Service Establishment Inspection,Routine,Approved mobile to operate. Will send copy of ...,1899-12-30T09:45:00.000Z,La Media Naranja LLC #2 Mobile FSE,7826 Santos Dr,,Murfreesboro,TN,37129,,53A4904B-BFF6-4E5F-BF43-79253A7BD020,Commercial Food <51 (Mobile),Food Service Establishment,605
2,stdh,C47AF1AD-1C7B-4AC4-9C4A-2D320A035257,2026-01-23T00:00:00.000Z,,100,Public Swimming Pools,Routine,,1899-12-30T09:10:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F9B53208-596A-40AB-AB59-D953C102B970,"Type D- Whirlpools, hot tubs",Public Swimming Pool,690
3,stdh,292318AF-BD49-4DE0-B891-E5E42590424E,2026-01-23T00:00:00.000Z,,100,Tattoo Studios Inspection,Routine,Establishment is very clean and organized!,1899-12-30T13:00:00.000Z,Got Ink? #615,8204 Florence Rd,,Smyrna,TN,37167,,F31C828F-71DC-4DDA-A56B-895DA3DFE22C,Tattoo Studios,Tattoo Studios,665
4,stdh,9C54FBC1-B6E3-4B22-816A-B01E8406E1A8,2026-01-23T00:00:00.000Z,,92,Food Service Establishment Inspection,Routine,,1899-12-30T13:34:00.000Z,Subway #2958,610 Dolly Parton Pkwy.,,Sevierville,TN,37862,,3691D738-F327-4327-8D16-7E5E28BB76BC,Commercial Food <51,Food Service Establishment,605


In [101]:
df['score'] = df['score'].astype('float64')

In [102]:
sorted_df = df.sort_values(by='score', ascending=False)

In [104]:
sorted_df[~sorted_df['score'].isna()]

,nick,inspectionID,inspectionDate,recommendation,score,inspectionType,purpose,comments,timein,establishmentName,addressLine1,addressLine2,city,state,zip,scoreDisplay,permitID,permitType,programName,programCode
0,stdh,F14D9D7C-6040-4579-ACCB-864AD185037F,2026-01-23T00:00:00.000Z,,100.0,Public Swimming Pools,Routine,,1899-12-30T09:00:00.000Z,Hamilton Family YMCA,7430 Shallowford Rd.,,Chattanooga,TN,37421,,F6556C9D-1982-419B-B96E-164D9F9831B7,Type A- General public and institutional pools,Public Swimming Pool,690
36,stdh,EEC5D6C1-9D93-4A87-A69C-188838E52CF7,2026-01-23T00:00:00.000Z,,100.0,Body Piercing Studios Inspection,Routine,See attached weekly spore test results in Heal...,1899-12-30T13:56:00.000Z,I Love You Piercing,143 Cherokee Heights Dr.,,Maryville,TN,37801,,D6289317-F03B-43CC-AB0B-EC9CE1F0AB6E,Body Piercing Studios,Body Piercing Studios,670
26,stdh,6331BA90-0DFA-43B2-8349-20488913C403,2026-01-23T00:00:00.000Z,,100.0,Hotels Motels Inspection,Follow-Up,Priority violation #18 fire extinguisher expir...,1899-12-30T12:10:00.000Z,CANDLEWOOD SUITES,270 VENTURE CIRCLE,,Nashville,TN,37228,,24EB5DFE-283D-4ACE-952A-2EEDE3227859,Hotel 51-150,Hotel,620
27,stdh,18D050CA-5BC6-4418-AF85-7682749998DF,2026-01-23T00:00:00.000Z,,100.0,Tattoo Studios Inspection,Routine,"areas checked: Jake ohlmann, Courtney golden, ...",1899-12-30T14:35:00.000Z,Outlier Tattoo,329 Donelson Pike,Ste 102,Nashville,TN,37214,,21E1BEE1-506F-48A2-B7E7-4648245E4199,Tattoo Studios,Tattoo Studios,665
67,stdh,C390BAD4-05A2-4472-8571-B074B12929FB,2026-01-23T00:00:00.000Z,,100.0,Food Service Establishment Inspection,Follow-Up,,1899-12-30T11:16:00.000Z,Liberty Creek Elementary Cafeteria,400 Wolfpack Way,,Gallatin,TN,37066,,1C2388C1-77E0-4627-9363-8D73AFC17E5E,School Cafeteria,Food Service Establishment,605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,stdh,B9EDDC51-1033-481D-AB5C-AC02A125E893,2026-01-23T00:00:00.000Z,,93.0,Food Service Establishment Inspection,Routine,,1899-12-30T10:16:00.000Z,Hardees Of Maynardville,2825 Maynardville Hwy.,,Maynardville,TN,37807,,15BBC94B-21F7-4DA5-BD66-457FBDC31B5F,Commercial Food 51+,Food Service Establishment,605
35,stdh,8B338D23-7EF5-41CD-9F32-7D590F0B5731,2026-01-23T00:00:00.000Z,,92.0,Food Service Establishment Inspection,Follow-Up,3rd repeat to be issued for item 8.,1899-12-30T16:25:00.000Z,Italia Pizza & Pasta,3803 Gallatin Rd.,,Nashville,TN,37216,,894896EB-35BE-4D69-9974-78720EC17739,Commercial Food <51,Food Service Establishment,605
4,stdh,9C54FBC1-B6E3-4B22-816A-B01E8406E1A8,2026-01-23T00:00:00.000Z,,92.0,Food Service Establishment Inspection,Routine,,1899-12-30T13:34:00.000Z,Subway #2958,610 Dolly Parton Pkwy.,,Sevierville,TN,37862,,3691D738-F327-4327-8D16-7E5E28BB76BC,Commercial Food <51,Food Service Establishment,605
60,stdh,65DDB3F8-3D47-44FB-8AC6-6D7D7DD3C9E1,2026-01-23T00:00:00.000Z,,91.0,Hotels Motels Inspection,Routine,Fire extinguishers UTD as of: Jan 2025. Inform...,1899-12-30T11:43:00.000Z,Holiday Inn Express & Suites,1112 Highway 321 N.,,Lenoir City,TN,37771,,5B6ABA5E-34E8-4010-8379-80F2C99E4606,Hotel 51-150,Hotel,620


### Summary 

- Check the query type, header and params using the developer tools 
- Often, multiple API queries are made to display one result 